In [3]:
import json as json
import numpy as np
import pandas as pd

In [4]:
import gensim

In [5]:
keyworddf = pd.read_json('nasa_keyword.json')

In [6]:
titledf = pd.read_json('nasa_title.json')

In [7]:
descdf = pd.read_json('nasa_description.json')

In [47]:
import gensim
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

def iter_wiki(dump_file):
    """Yield each article from the Wikipedia dump, as a `(title, tokens)` 2-tuple."""
    ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for title, text, pageid in _extract_pages(smart_open(dump_file)):
        text = filter_wiki(text)
        tokens = tokenize(text)
        if len(tokens) < 50 or any(title.startswith(ns + ':') for ns in ignore_namespaces):
            continue  # ignore short articles and various meta-articles
        yield title, tokens

def rec_topic(doc, id2word, model):
    bow = id2word.doc2bow(tokenize(doc))
    doc_lda = model[bow]
    try:
        print(model.print_topic(max(doc_lda, key=lambda item: item[1])[0]))
    except Error as e:
        print (e.strerror)

In [9]:
import bz2
id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_wordids.txt.bz2')
model = gensim.models.LdaModel.load('myldamodel.lda')

## Example topic recommender

In [36]:
# Example description text
example_desc = descdf.description[0]
print (example_desc)

USGS 15 minute stream flow data for Kings Creek on the Konza Prairie


In [37]:
# Example topics recommended for description
rec_topic(example_desc, id2word, model)

0.010*river + 0.008*lake + 0.006*creek + 0.005*park + 0.005*island + 0.005*mountain + 0.004*water + 0.004*reserve + 0.004*forest + 0.003*dam


In [38]:
print(titledf.title[0])

15 Minute Stream Flow Data: USGS (FIFE)


In [39]:
print(keyworddf.keyword[0])

[u'EARTH SCIENCE', u'HYDROSPHERE', u'SURFACE WATER']


In [48]:
num_rows = descdf.description.count()
print(num_rows)

import random
random_indices = random.sample(xrange(num_rows), 10)
print random_indices

for i in random_indices: 
    desc_text = descdf.description[i]
    title_text = titledf.title[i]
    keywords = keyworddf.keyword[i]
    print ('Description:')
    print (desc_text)
    rec_topic(desc_text, id2word, model)
    print ('\nTitle:')
    print (title_text)
    rec_topic(title_text, id2word, model)
    print ('\nKeywords')
    print (keywords)
    print ('\n\n')

32089
[19069, 11524, 7004, 6690, 15, 20227, 642, 14741, 6327, 20322]
Description:
This data set contains a series of land surface parameters simulated from the VIC land-surface model (LSM) for Phase 2 of the North American Land Data Assimilation System (NLDAS-2). The data are in 1/8th degree grid spacing and range from Jan 1979 to the present. The temporal resolution is hourly. The file format is WMO GRIB-1.

Details about the NLDAS-2 configuration of the VIC LSM can be found in Xia et al. (2012). The version of the VIC model for the NLDAS-2 VIC data available from the NASA GES DISC is VIC-4.0.3; this version of the VIC model is the same as used in Sheffield et al. (2003). 

The VIC model was developed at the University of Washington and Princeton University as a macroscale, semi-distributed, grid-based, hydrologic model [Liang et al., 1994; Wood et al., 1997].  The full water and energy balance modes of VIC were used for NLDAS-2. VIC uses three soil layers, with thicknesses that vary 